In [7]:
# 8. b^4 - 6a + 5b^2 + 2a - 3b^4

class Operation():                  
    
    def __init__(self, input_nodes = []):
        self.input_nodes = input_nodes # The list of input nodes
        self.output_nodes = []
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
        
    def compute(self):
        pass
    
class add(Operation):
    
    def __init__(self, x, y):
         
        super().__init__([x, y])

    def compute(self, x_var, y_var):
         
        self.inputs = [x_var, y_var]
        return x_var + y_var
    
class multiply(Operation):
     
    def __init__(self, a, b):
        
        super().__init__([a, b])
    
    def compute(self, a_var, b_var):
         
        self.inputs = [a_var, b_var]
        return a_var * b_var
    
class Placeholder():
    
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)
        
class Variable():
    
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
         
        _default_graph.variables.append(self)
        
class Graph():
    
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        """
        Sets this Graph instance as the Global Default Graph
        """
        global _default_graph
        _default_graph = self

graph = Graph()
graph.set_as_default()
x = Variable(-6) #b^4 - 6a + 5b^2 + 2a - 3b^4
y = Variable(5)
c = Variable(2)
d = Variable(-3)

a = Placeholder()
b = Placeholder()

r = multiply(b, b) #b^2
s = multiply(r, b) #b^3
t = multiply(s, b) #b^4
f = multiply(x, a) #-6a
g = add(t, f) #b^4-6a
h = multiply(r, y) #5b^2
i = add(g, h) #b^4-6a+5b^2
j = multiply(a, c) #2a
k = add(j, i) #b^4 - 6a + 5b^2 + 2a
l = multiply(d, t)  #-3b^4   
m = add(l, k)     ##b^4 - 6a + 5b^2 + 2a - 3b^4

import numpy as np

def traverse_postorder(operation):
    """ 
    PostOrder Traversal of Nodes. Basically makes sure computations are done in 
    the correct order (Ax first , then Ax + b). Feel free to copy and paste this code.
    It is not super important for understanding the basic fundamentals of deep learning.
    """
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder


class Session:
    
    def run(self, operation, feed_dict = {}):
        """ 
          operation: The operation to compute
          feed_dict: Dictionary mapping placeholders to input values (the data)  
        """
       
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:

            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else: # Operation
                
                node.inputs = [input_node.output for input_node in node.input_nodes]

                 
                node.output = node.compute(*node.inputs)
                
            # Convert lists to numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        # Return the requested node value
        return operation.output



In [8]:
session = Session()
result = session.run(operation=m, feed_dict = {a:[2, 4, 6, 8], b:[6, 7, 8, 9]})
result

array([ -2420,  -4573,  -7896, -12749])